## <b>混和重加密
1. 對稱加密數據：
使用對稱密鑰（例如AES密鑰）對數據進行加密，生成密文。

2. 加密對稱密鑰：
使用非對稱加密算法（例如RSA）對對稱密鑰進行加密，生成加密對稱密鑰。

3. 重加密對稱密鑰：
當需要將加密數據重新加密給另一個接收者時，將加密對稱密鑰使用接收者的公鑰重新加密。

4. 傳輸和解密：
接收者接收到加密數據和加密對稱密鑰後，使用自己的私鑰解密對稱密鑰，然後使用解密的對稱密鑰來解密數據。

In [1]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import os

# 生成RSA密鑰對
def generate_rsa_keys():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()
    return private_key, public_key

# 對稱加密數據
def encrypt_data_symmetric(key, data):
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    encrypted_data = encryptor.update(data) + encryptor.finalize()
    return iv + encrypted_data

# 對稱解密數據
def decrypt_data_symmetric(key, encrypted_data):
    iv = encrypted_data[:16]
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    data = decryptor.update(encrypted_data[16:]) + decryptor.finalize()
    return data

# 使用公鑰加密對稱密鑰
def encrypt_symmetric_key(public_key, symmetric_key):
    encrypted_key = public_key.encrypt(
        symmetric_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return encrypted_key

# 使用私鑰解密對稱密鑰
def decrypt_symmetric_key(private_key, encrypted_key):
    symmetric_key = private_key.decrypt(
        encrypted_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return symmetric_key

# 生成RSA密鑰對
private_key_1, public_key_1 = generate_rsa_keys()
private_key_2, public_key_2 = generate_rsa_keys()

# 生成對稱密鑰
symmetric_key = os.urandom(32)

# 原始數據
data = b"Secret data that needs to be encrypted."

# 使用對稱密鑰加密數據
encrypted_data = encrypt_data_symmetric(symmetric_key, data)

# 使用公鑰加密對稱密鑰
encrypted_symmetric_key = encrypt_symmetric_key(public_key_1, symmetric_key)

# 模擬代理重加密對稱密鑰給新接收者
re_encrypted_symmetric_key = encrypt_symmetric_key(public_key_2, decrypt_symmetric_key(private_key_1, encrypted_symmetric_key))

# 新接收者解密對稱密鑰
final_symmetric_key = decrypt_symmetric_key(private_key_2, re_encrypted_symmetric_key)

# 新接收者解密數據
final_decrypted_data = decrypt_data_symmetric(final_symmetric_key, encrypted_data)

print(f"Original Data: {data}")
print(f"Decrypted Data: {final_decrypted_data}")

Original Data: b'Secret data that needs to be encrypted.'
Decrypted Data: b'Secret data that needs to be encrypted.'


## <b>代理重加密

1.生成RSA密鑰對：生成數據擁有者和接收者的公私鑰對。
2.對稱加密數據：使用對稱密鑰加密數據。
3.加密對稱密鑰：使用數據擁有者的公鑰對對稱密鑰進行加密。
4.生成重加密密鑰：數據擁有者生成一個重加密密鑰，用於代理進行重加密操作。
5.代理重加密：代理使用重加密密鑰對加密的對稱密鑰進行重加密。
6.接收者解密和使用：接收者使用其私鑰解密對稱密鑰，並使用該對稱密鑰解密數據。

不需要代理方擁有數據擁有者的私鑰，並且數據在傳輸過程中始終保持加密狀態。

In [2]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import os

# 生成RSA密鑰對
def generate_rsa_keys():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()
    return private_key, public_key

# 對稱加密數據
def encrypt_data_symmetric(key, data):
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    encrypted_data = encryptor.update(data) + encryptor.finalize()
    return iv + encrypted_data

# 對稱解密數據
def decrypt_data_symmetric(key, encrypted_data):
    iv = encrypted_data[:16]
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    data = decryptor.update(encrypted_data[16:]) + decryptor.finalize()
    return data

# 使用公鑰加密對稱密鑰
def encrypt_symmetric_key(public_key, symmetric_key):
    encrypted_key = public_key.encrypt(
        symmetric_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return encrypted_key

# 使用私鑰解密對稱密鑰
def decrypt_symmetric_key(private_key, encrypted_key):
    symmetric_key = private_key.decrypt(
        encrypted_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return symmetric_key

# 重加密密鑰生成（模擬，實際中需使用專門的重加密算法）
def generate_re_encryption_key(private_key_1, public_key_2):
    # 這裡使用簡單的方式模擬生成重加密密鑰
    return (private_key_1, public_key_2)

# 使用重加密密鑰進行重加密（模擬，實際中需使用專門的重加密算法）
def re_encrypt_key(re_encryption_key, encrypted_symmetric_key):
    private_key_1, public_key_2 = re_encryption_key
    symmetric_key = decrypt_symmetric_key(private_key_1, encrypted_symmetric_key)
    re_encrypted_key = encrypt_symmetric_key(public_key_2, symmetric_key)
    return re_encrypted_key

# 生成RSA密鑰對
private_key_1, public_key_1 = generate_rsa_keys()
private_key_2, public_key_2 = generate_rsa_keys()

# 生成對稱密鑰
symmetric_key = os.urandom(32)

# 原始數據
data = b"Secret data that needs to be encrypted."

# 使用對稱密鑰加密數據
encrypted_data = encrypt_data_symmetric(symmetric_key, data)

# 使用數據擁有者的公鑰加密對稱密鑰
encrypted_symmetric_key = encrypt_symmetric_key(public_key_1, symmetric_key)

# 生成重加密密鑰（由數據擁有者生成）
re_encryption_key = generate_re_encryption_key(private_key_1, public_key_2)

# 代理使用重加密密鑰對對稱密鑰進行重加密
re_encrypted_symmetric_key = re_encrypt_key(re_encryption_key, encrypted_symmetric_key)

# 接收者使用其私鑰解密對稱密鑰
final_symmetric_key = decrypt_symmetric_key(private_key_2, re_encrypted_symmetric_key)

# 接收者使用對稱密鑰解密數據
final_decrypted_data = decrypt_data_symmetric(final_symmetric_key, encrypted_data)

print(f"Original Data: {data}")
print(f"Decrypted Data: {final_decrypted_data}")

Original Data: b'Secret data that needs to be encrypted.'
Decrypted Data: b'Secret data that needs to be encrypted.'


In [6]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.backends import default_backend
import os

# 生成RSA密鑰對
def generate_rsa_keys():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()
    return private_key, public_key

# 使用私鑰和隨機數加密數據
def encrypt_data_with_nonce(private_key, data, nonce):
    encrypted_data = private_key.public_key().encrypt(
        data + nonce,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return encrypted_data

# 使用私鑰解密數據，不需要nonce
def decrypt_data(private_key, encrypted_data):
    decrypted_data = private_key.decrypt(
        encrypted_data,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return decrypted_data

# 使用私鑰和隨機數解密數據，並驗證隨機數
def decrypt_data_with_nonce(private_key, encrypted_data, nonce):
    decrypted_data = decrypt_data(private_key, encrypted_data)
    data, nonce_in_decrypted = decrypted_data[:-len(nonce)], decrypted_data[-len(nonce):]
    assert nonce == nonce_in_decrypted, "Nonce does not match!"
    return data

# 模擬生成重加密密鑰
def generate_re_encryption_key(private_key_1, public_key_2):
    return (private_key_1, public_key_2)

# 使用重加密密鑰進行重加密
def re_encrypt_data(re_encryption_key, encrypted_data):
    private_key_1, public_key_2 = re_encryption_key
    data_with_nonce = decrypt_data(private_key_1, encrypted_data)  # 不需要nonce
    re_encrypted_data = public_key_2.encrypt(
        data_with_nonce,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return re_encrypted_data

# 生成RSA密鑰對
private_key_1, public_key_1 = generate_rsa_keys()
private_key_2, public_key_2 = generate_rsa_keys()

# 原始文件數據
data = b"Secret file data that needs to be encrypted and re-encrypted."
nonce = os.urandom(16)  # 生成隨機數

# 使用者使用其私鑰和隨機數加密數據
encrypted_data = encrypt_data_with_nonce(private_key_1, data, nonce)

# 存儲加密的數據和隨機數
stored_encrypted_data = encrypted_data
stored_nonce = nonce

# 生成重加密密鑰（由使用者生成）
re_encryption_key = generate_re_encryption_key(private_key_1, public_key_2)

# 代理使用重加密密鑰對數據進行重加密
re_encrypted_data = re_encrypt_data(re_encryption_key, stored_encrypted_data)

# 接收者使用其私鑰和隨機數解密數據
final_data = decrypt_data_with_nonce(private_key_2, re_encrypted_data, stored_nonce)

print(f"Original data: {data}")
print(f"Final decrypted data: {final_data}")


Original data: b'Secret file data that needs to be encrypted and re-encrypted.'
Final decrypted data: b'Secret file data that needs to be encrypted and re-encrypted.'
